# Laboratoire 2 : Arbre de désision, Bayes naïf et KNN
#### Département du génie logiciel et des technologies de l’information

| Étudiants             | Alexendre Bleau — BLEA14058906   David Létinaud  — LETD05129708  Thomas Lioret   — LIOT20069605|
|-----------------------|---------------------------------------------------------|
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Automne 2019                                            |
| Groupe                | 1                                                       |
| Numéro du laboratoire | 1                                                       |
| Professeur            | Prof. LOMBAERT                                          |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 27/10/2019                                              |

In [ ]:
import numpy as np

## Question 1
### Méthode de création des ensembles de données
Pour éviter de recalculer plusieurs fois les features, il peut être plus efficasse de les enregistrer pour pouvoir les réutiliser par la suite.
On va par exemple enregistrer les features utilisées dans le TP1 dans un fichier csv. Pour cela on utilise la bibliothèque 'csv' et on ouvre un fichier en écriture. On utilisera ensuite la fonction 'writerows' pour écrire dans ce fichier une list en format compatible csv.

## Question 1
### Détails des ensembles produits
L'ensemble de données ainsi produit contient toutes les informations utiles pour de l'apprentissage supervisée et sont disposées de cette façon :
['numero_image', 'feature_1','feature_2','feature_3', 'classe_image']

## Question 2
### Approche de validation proposée et justification

## Question 3
### Matrice des expérimentations

## Question 3
### Étude des hyperparamètres et des modèles

## Question 4
### Impact de la taille des ensembles de données sur la performance de classification

## Question 5
### Impact du bruit dans les ensembles de données sur la performance de classification

## Question 6
### Discussion sur la nature des données

## Question 7
### Recommandations

## Question 8
### Améliorations possibles